In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import tensorflow.keras.layers as layers

In [2]:
data = pd.read_csv(r"C:\Users\zubik\CODIGOS\jupyter\machine learning\titanic\data\train.csv")

In [3]:
X = data.copy()
y = X.pop("Survived")
X_test = pd.read_csv(r"C:\Users\zubik\CODIGOS\jupyter\machine learning\titanic\data\test.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

def pd_transform(data, y_data = 0):
    data.drop(["Name", "Ticket","Cabin","PassengerId"],axis=1,inplace=True)
    data['Age'].fillna(30, inplace=True)
    rows_dropped = data[data["Embarked"].isnull()].index

    data.drop(rows_dropped, inplace=True)

    data['Family'] = data.SibSp + data.Parch
    data.drop(['SibSp','Parch'],axis=1,inplace=True)

    data['Sex'].replace({'male':0,'female':1}, inplace=True)

    if isinstance(y_data, pd.Series):
        y_data.drop(rows_dropped,inplace=True)
    
    onehot = pd.DataFrame(OneHotEncoder(sparse=False).fit_transform(data[['Embarked']]))

    onehot.reset_index(inplace=True)
    data.reset_index(inplace=True)
    data = pd.concat([data, onehot], axis=1)

    data.drop(['Embarked', 'index'],axis=1, inplace=True)
    data.columns = data.columns.map(str)
    data = pd.DataFrame( StandardScaler().fit_transform(data) )
    
    return data, y_data

In [5]:
X, y = pd_transform(X, y)

In [6]:
X.shape

(889, 8)

In [7]:
X.head(4)

,0,1,2,3,4,5,6,7
0,0.825209,-0.735342,-0.595082,-0.500240,0.057853,-0.482711,-0.307941,0.616794
1,-1.572211,1.359911,0.639311,0.788947,0.057853,2.071634,-0.307941,-1.621287
2,0.825209,1.359911,-0.286483,-0.486650,-0.561804,-0.482711,-0.307941,0.616794
3,-1.572211,1.359911,0.407863,0.422861,0.057853,-0.482711,-0.307941,0.616794


First version: without dropout

In [8]:
model = keras.Sequential([
    layers.Dense(10, input_shape=[8], activation='relu'),
    layers.Dense(10,activation="relu"),
    layers.Dense(5, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer="adam", metrics=["accuracy"])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=16)

In [12]:
X_train.shape

(800, 8)

In [13]:
model.fit(X,y,validation_split=0.2, epochs=30, batch_size= 90)

Epoch 1/30
8/8 [==============================] - 1s 28ms/step - loss: 0.7022 - accuracy: 0.6160 - val_loss: 0.6797 - val_accuracy: 0.6685
Epoch 2/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6855 - accuracy: 0.6456 - val_loss: 0.6643 - val_accuracy: 0.6910
Epoch 3/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6714 - accuracy: 0.6596 - val_loss: 0.6492 - val_accuracy: 0.7247
Epoch 4/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6575 - accuracy: 0.6709 - val_loss: 0.6343 - val_accuracy: 0.7416
Epoch 5/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6451 - accuracy: 0.6807 - val_loss: 0.6206 - val_accuracy: 0.7416
Epoch 6/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6330 - accuracy: 0.6948 - val_loss: 0.6076 - val_accuracy: 0.7472
Epoch 7/30
8/8 [==============================] - 0s 3ms/step - loss: 0.6222 - accuracy: 0.7187 - val_loss: 0.5961 - val_accuracy: 0.7584
Epoch 8/30
8/8 [=================

Fitting with whole data

In [14]:
model.fit(X, y, epochs=30)

Epoch 1/30
28/28 [==============================] - 0s 815us/step - loss: 0.4506 - accuracy: 0.8166
Epoch 2/30
28/28 [==============================] - 0s 741us/step - loss: 0.4416 - accuracy: 0.8121
Epoch 3/30
28/28 [==============================] - 0s 741us/step - loss: 0.4366 - accuracy: 0.8189
Epoch 4/30
28/28 [==============================] - 0s 778us/step - loss: 0.4311 - accuracy: 0.8211
Epoch 5/30
28/28 [==============================] - 0s 778us/step - loss: 0.4304 - accuracy: 0.8166
Epoch 6/30
28/28 [==============================] - 0s 741us/step - loss: 0.4253 - accuracy: 0.8200
Epoch 7/30
28/28 [==============================] - 0s 741us/step - loss: 0.4233 - accuracy: 0.8223
Epoch 8/30
28/28 [==============================] - 0s 815us/step - loss: 0.4209 - accuracy: 0.8189
Epoch 9/30
28/28 [==============================] - 0s 741us/step - loss: 0.4178 - accuracy: 0.8256
Epoch 10/30
28/28 [==============================] - 0s 778us/step - loss: 0.4149 - accuracy: 0.8290

In [15]:
X_test,_ = pd_transform(X_test)

In [16]:
X_test.fillna(7000,inplace=True)

In [17]:
preds = model.predict(X_test)

In [18]:
s = preds.reshape(1,-1)

In [19]:
index  =pd.read_csv(r"C:\Users\zubik\CODIGOS\jupyter\machine learning\titanic\data\test.csv").PassengerId
output = pd.DataFrame({'PassengerId': index,
                       'Survived': (s[0] > 0.5).astype(int) })
output.to_csv('./submissionv1.csv', index=False)

Second version: with dropout

In [20]:
model2 = keras.Sequential([
        layers.Dense(10, input_shape=[8], activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10,activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(5, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(1, activation='sigmoid')
])

In [21]:
e_s = keras.callbacks.EarlyStopping(min_delta=0.001, patience=5, restore_best_weights=True)

In [22]:
model2.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.binary_accuracy])
model2.fit(X, y, validation_split=0.1, epochs=100, callbacks=[e_s])

Epoch 1/100
25/25 [==============================] - 0s 5ms/step - loss: 0.6808 - binary_accuracy: 0.6675 - val_loss: 0.6694 - val_binary_accuracy: 0.7079
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6688 - binary_accuracy: 0.6787 - val_loss: 0.6577 - val_binary_accuracy: 0.7303
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.6605 - binary_accuracy: 0.6687 - val_loss: 0.6417 - val_binary_accuracy: 0.7416
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 0.6356 - binary_accuracy: 0.6913 - val_loss: 0.6056 - val_binary_accuracy: 0.7528
Epoch 5/100
25/25 [==============================] - 0s 1ms/step - loss: 0.5991 - binary_accuracy: 0.7163 - val_loss: 0.5544 - val_binary_accuracy: 0.7865
Epoch 6/100
25/25 [==============================] - 0s 1ms/step - loss: 0.5854 - binary_accuracy: 0.7113 - val_loss: 0.5227 - val_binary_accuracy: 0.7865
Epoch 7/100
25/25 [==============================] - 0s 1ms/step - los

In [23]:
preds2 = model2.predict(X_test).round()
preds2 = preds2.reshape(1,-1)

In [24]:
index  =pd.read_csv(r"C:\Users\zubik\CODIGOS\jupyter\machine learning\titanic\data\test.csv").PassengerId
output = pd.DataFrame({'PassengerId': index,
                       'Survived': preds2[0].astype(int) })
output.to_csv('./submissionv2.csv', index=False)